In [ ]:
import os
import sys

# Clone the SoccerTrack repository
if not os.path.isdir("/content/SoccerTrack"):
    print("Cloning SoccerTrack repository...")
    os.system("git clone https://github.com/JulezKlein/SoccerTrack.git /content/SoccerTrack")
    print("✓ Repository cloned")
else:
    print("✓ SoccerTrack repository already exists")

# Change to repo directory
os.chdir("/content/SoccerTrack")
sys.path.insert(0, "/content/SoccerTrack")

In [ ]:
# Configuration
DATASET = "football"  # Change to "soccertrack" if using that dataset
VIEW_TYPE = "top_view"  # Only relevant for soccertrack ("top_view" or "wide_view")
MODEL = "rtdetr-l"  # Options: yolov8s, yolov8n, yolo26n, rtdetr-l, rtdetr-x
EPOCHS = 70
IMG_SIZE = 640

# Mount Google Drive (for Colab data access)
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✓ Google Drive mounted")
    DATA_DIR = "/content/drive/MyDrive/datasets"
except:
    print("⚠ Google Colab not detected - running in local environment")
    DATA_DIR = "./data"

In [ ]:
import torch

print("\n" + "="*60)
print("Environment Setup")
print("="*60)
print(f"PyTorch version: {torch.__version__}")
device = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Device available: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA available: True")
    os.system("nvidia-smi")
else:
    print("CUDA available: False")

In [ ]:
# Install dependencies
print("\n" + "="*60)
print("Installing Dependencies")
print("="*60)

import subprocess
import sys

subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "pip", "-q"], check=True)
subprocess.run(
    [sys.executable, "-m", "pip", "install", 
     "ultralytics", "pandas", "pillow", "tqdm", "opencv-python", "matplotlib", "-q"],
    check=True
)
print("✓ Dependencies installed")

In [ ]:
# Import training functions from the script
from train_ultralytics_models import (
    configure_dataset,
    unzip_dataset,
    prepare_dataset,
    validate_dataset,
    visualize_sample,
    start_training
)

# Configure dataset
print("\n" + "="*60)
print(f"Configuring Dataset: {DATASET.upper()}")
print("="*60)
config = configure_dataset(dataset_type=DATASET, view_type=VIEW_TYPE)

In [ ]:
# Prepare dataset (only if needed for SoccerTrack)
print("\n" + "="*60)
print("Dataset Preparation")
print("="*60)

if config["requires_preparation"]:
    print(f"Preparing {DATASET} dataset for training...")
    unzip_dataset()
    prepare_dataset()
    print("✓ Dataset preparation complete")
else:
    print(f"✓ {DATASET} dataset is already in expected format")
    print("  No preparation needed - data should be uploaded to:")
    print(f"  {config['dataset_dir']}")

In [ ]:
# Visualize sample from dataset
print("\n" + "="*60)
print("Visualizing Sample")
print("="*60)

try:
    visualize_sample()
    print("✓ Sample visualization complete")
except Exception as e:
    print(f"⚠ Could not visualize sample: {e}")

In [ ]:
# Validate dataset before training
print("\n" + "="*60)
print("Validating Dataset Format")
print("="*60)

try:
    validate_dataset()
    print("✓ Dataset validation successful")
except Exception as e:
    print(f"⚠ Validation warning: {e}")
    print("  Proceeding with training anyway...")

In [ ]:
# Print training configuration
print("\n" + "="*60)
print("Training Configuration")
print("="*60)
print(f"Dataset: {DATASET}")
if DATASET == "soccertrack":
    print(f"View Type: {VIEW_TYPE}")
print(f"Model: {MODEL}")
print(f"Epochs: {EPOCHS}")
print(f"Image Size: {IMG_SIZE}")
print(f"Device: {device}")
print(f"Output Dir: {config['output_dir']}")
print("="*60)

In [ ]:
# Start training
print("\n" + "="*60)
print(f"Starting {MODEL.upper()} Training")
print("="*60)

try:
    results = start_training(
        epochs=EPOCHS,
        img_size=IMG_SIZE,
        model_name=MODEL
    )
    print("\n✅ Training completed successfully!")
    print(f"Results saved to: {config['output_dir']}/{MODEL}")
except Exception as e:
    print(f"\n❌ Training failed: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Optional: Check training results
import glob
from pathlib import Path

output_dir = Path(config['output_dir']) / MODEL
if output_dir.exists():
    print("\n" + "="*60)
    print("Training Results")
    print("="*60)
    
    # List results directories
    result_dirs = list(output_dir.glob("train*"))
    if result_dirs:
        latest_run = max(result_dirs, key=lambda p: p.stat().st_mtime)
        print(f"Latest training run: {latest_run.name}")
        
        # List key files
        weights_dir = latest_run / "weights"
        if weights_dir.exists():
            print(f"\nWeights:")
            for weight_file in weights_dir.glob("*.pt"):
                print(f"  - {weight_file.name}")
        
        # Check for results.csv
        results_csv = latest_run / "results.csv"
        if results_csv.exists():
            print(f"\n✓ Results logged to: {results_csv}")
    else:
        print("No completed training runs found yet")

In [ ]:
# Optional: Download trained model from Colab
try:
    from google.colab import files
    
    weights_path = output_dir / "train" / "weights" / "best.pt"
    if weights_path.exists():
        print(f"\nDownloading trained model: {weights_path.name}")
        files.download(str(weights_path))
        print("✓ Model download started")
    else:
        print("No trained weights found to download")
except ImportError:
    print("(Google Colab environment not detected - skipping download)")

In [ ]:
# ============================================================
# EXPORT CONFIGURATION
# ============================================================
# Choose export format(s): "coreml", "onnx", or "both"
EXPORT_FORMAT = "coreml"  # Change to "onnx" or "both" as needed
EXPORT_FP16 = True  # Use FP16 precision (False for FP32)
DISABLE_NMS = False  # Set True to disable NMS in exported model

CONF_THRESH = 0.4  # Confidence threshold baked into model
IOU_THRESH = 0.2   # IoU threshold baked into model

# Find the best weights from training
from pathlib import Path
import glob

output_path = Path(config['output_dir']) / MODEL
result_dirs = sorted(output_path.glob("train*"), key=lambda p: p.stat().st_mtime, reverse=True)
if result_dirs:
    WEIGHTS = str(result_dirs[0] / "weights" / "best.pt")
    print(f"Found trained weights: {WEIGHTS}")
else:
    WEIGHTS = f"runs/detect/output_yolo_football/{MODEL}/weights/best.pt"
    print(f"Using default weights path: {WEIGHTS}")
    
print(f"\nExport Configuration:")
print(f"  Format: {EXPORT_FORMAT}")
print(f"  FP16: {EXPORT_FP16}")
print(f"  NMS: {not DISABLE_NMS}")
print(f"  Conf Threshold: {CONF_THRESH}")
print(f"  IoU Threshold: {IOU_THRESH}")

In [ ]:
# ============================================================
# EXPORT FUNCTIONS
# ============================================================
import os
import time
from PIL import Image
import numpy as np
from ultralytics import YOLO

def export_coreml():
    """Export model to Core ML format"""
    print("🔄 Loading YOLO model...")
    model = YOLO(WEIGHTS)

    print("📦 Exporting to Core ML...")
    try:
        model.export(
            format="coreml",
            imgsz=IMG_SIZE,
            nms=not DISABLE_NMS,
            half=EXPORT_FP16,
            conf=CONF_THRESH,
            iou=IOU_THRESH,
            data=config['yaml_path']
        )
        print("✅ Core ML export finished")
        return True
    except Exception as e:
        print(f"❌ Core ML export failed: {e}")
        return False


def export_onnx():
    """Export model to ONNX format"""
    print("🔄 Loading YOLO model...")
    model = YOLO(WEIGHTS)

    print("📦 Exporting to ONNX...")
    try:
        model.export(
            format="onnx",
            imgsz=IMG_SIZE,
            nms=not DISABLE_NMS,
            end2end=True,
            conf=CONF_THRESH,
            iou=IOU_THRESH,
            half=EXPORT_FP16,
            data=config['yaml_path']
        )
        print("✅ ONNX export finished")
        return True
    except Exception as e:
        print(f"❌ ONNX export failed: {e}")
        return False


def quick_test_coreml():
    """Test Core ML model with dummy input"""
    print("\n🧪 Running Core ML inference test...")
    
    try:
        import coremltools as ct
    except ImportError:
        print("⚠️ coremltools not installed. Installing...")
        os.system("pip install coremltools -q")
        import coremltools as ct

    mlpackage_path = WEIGHTS.replace(".pt", ".mlpackage")

    if not os.path.exists(mlpackage_path):
        print(f"⚠️ Core ML model not found at {mlpackage_path}")
        return

    try:
        mlmodel = ct.models.MLModel(mlpackage_path)
        print("✓ Core ML model loaded successfully")
    except Exception as e:
        print(f"❌ Failed to load Core ML model: {e}")
        return

    dummy_input = Image.new(mode="RGB", size=(IMG_SIZE, IMG_SIZE), color=(128, 128, 128))

    try:
        outputs = mlmodel.predict({"image": dummy_input})
        print("✅ Core ML inference successful")
        print("📤 Output keys:")
        for k, v in outputs.items():
            try:
                print(f"  - {k}: {v.shape}")
            except:
                print(f"  - {k}: (non-array output)")
    except Exception as e:
        print(f"❌ Core ML inference failed: {e}")
        return

    try:
        t0 = time.perf_counter()
        _ = mlmodel.predict({"image": dummy_input})
        t1 = time.perf_counter()
        elapsed_ms = (t1 - t0) * 1000.0
        print(f"⏱️ Core ML inference time: {elapsed_ms:.2f} ms")
    except Exception as e:
        print(f"❌ Core ML timing failed: {e}")


def quick_test_onnx():
    """Test ONNX model with dummy input"""
    print("\n🧪 Running ONNX runtime inference test...")

    try:
        import onnxruntime as ort
    except ImportError:
        print("⚠️ onnxruntime not installed. Installing...")
        os.system("pip install onnxruntime -q")
        import onnxruntime as ort

    onnx_path = WEIGHTS.replace(".pt", ".onnx")
    
    if not os.path.exists(onnx_path):
        print(f"⚠️ ONNX model not found at {onnx_path}")
        return

    try:
        sess = ort.InferenceSession(onnx_path)
        print("✓ ONNX model loaded successfully")
        
        input_name = sess.get_inputs()[0].name
        dummy = np.random.rand(1, 3, IMG_SIZE, IMG_SIZE).astype(np.float32)
        
        # Warm-up
        sess.run(None, {input_name: dummy})
        
        # Timed run
        t0 = time.perf_counter()
        outputs = sess.run(None, {input_name: dummy})
        t1 = time.perf_counter()
        elapsed_ms = (t1 - t0) * 1000.0
        
        print("✅ ONNX runtime inference successful")
        print(f"⏱️ ONNX inference time: {elapsed_ms:.2f} ms")
        print("📤 Output tensors:")
        for i, out in enumerate(outputs):
            print(f"  - output[{i}]: shape={getattr(out, 'shape', 'unknown')}")
    except Exception as e:
        print(f"❌ ONNX runtime test failed: {e}")


print("✓ Export functions defined")

In [ ]:
# ============================================================
# RUN EXPORTS
# ============================================================
print("\n" + "="*60)
print("Model Export")
print("="*60)

fmt = EXPORT_FORMAT.strip().lower() if isinstance(EXPORT_FORMAT, str) else "coreml"

if fmt == "coreml":
    if export_coreml():
        quick_test_coreml()
elif fmt == "onnx":
    if export_onnx():
        quick_test_onnx()
elif fmt == "both":
    print("\n📦 Exporting to both formats...\n")
    if export_coreml():
        quick_test_coreml()
    print("\n" + "-"*60 + "\n")
    if export_onnx():
        quick_test_onnx()
else:
    print(f"❌ Unsupported EXPORT_FORMAT: {EXPORT_FORMAT}")
    print("   Choose 'coreml', 'onnx', or 'both'")

print("\n✅ Export pipeline complete!")

In [ ]:
# ============================================================
# DOWNLOAD EXPORTED MODELS (Colab only)
# ============================================================
try:
    from google.colab import files
    
    print("\n" + "="*60)
    print("Downloading Exported Models")
    print("="*60)
    
    files_to_download = []
    
    if fmt in ["coreml", "both"]:
        mlpackage_path = WEIGHTS.replace(".pt", ".mlpackage")
        if os.path.exists(mlpackage_path):
            print(f"📦 Found Core ML model: {os.path.basename(mlpackage_path)}")
            files_to_download.append(mlpackage_path)
    
    if fmt in ["onnx", "both"]:
        onnx_path = WEIGHTS.replace(".pt", ".onnx")
        if os.path.exists(onnx_path):
            print(f"📦 Found ONNX model: {os.path.basename(onnx_path)}")
            files_to_download.append(onnx_path)
    
    # Also download the best.pt weights
    pt_weights = WEIGHTS
    if os.path.exists(pt_weights):
        print(f"📦 Found PyTorch weights: {os.path.basename(pt_weights)}")
        files_to_download.append(pt_weights)
    
    if files_to_download:
        print(f"\n📥 Downloading {len(files_to_download)} file(s)...")
        for file_path in files_to_download:
            print(f"  → {os.path.basename(file_path)}")
            files.download(file_path)
        print("✅ Download complete!")
    else:
        print("⚠️ No exported models found to download")
        
except ImportError:
    print("\n" + "="*60)
    print("Export Summary")
    print("="*60)
    print("✓ Google Colab environment not detected")
    print("  (Skipping automatic download)")
    print(f"\n📁 Exported model location:")
    print(f"  {os.path.dirname(WEIGHTS)}")